# Adding Visual Features

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.remove('snorkel.db')

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

from pdb import set_trace as t

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step 1: PDF to HTML Conversion

Use Adobe Acrobat (or other program of your choice) to convert PDF -> HTML with structure.

In [2]:
from visual_linking import VisualLinker
vizlink = VisualLinker(session)

### Step 2: Parse PDF

In [ ]:
pdf_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_pdf/'
filename = 'bc546-d'
pdf_file = pdf_path + filename + '.pdf' # Path to PDF file 

In [ ]:
%time vizlink.extract_pdf_words(pdf_file)

> /Users/bradenhancock/snorkel/tutorials/tables/visual_features/visual_linking.py(23)extract_pdf_words()
-> pdf_word_list_i, coordinate_map_i = self._coordinates_from_HTML(html_content, i)
(Pdb) print html_content
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>BC546 - NPN Amplifier Transistors</title>
<meta name="Author" content="s2190c"/>
<meta name="Creator" content="BroadVision, Inc."/>
<meta name="Producer" content="Acrobat Distiller 9.5.1 (Windows)"/>
<meta name="CreationDate" content=""/>
</head>
<body>
<doc>
  <page width="612.000000" height="792.000000">
    <flow>
      <block xMin="59.760000" yMin="50.100000" xMax="286.147800" yMax="93.420600">
        <line xMin="59.760000" yMin="50.100000" xMax="286.147800" yMax="73.500000">
          <word xMin="59.760000" yMin="50.100000" xMax="143.805600" yMax="73.500000">BC546B,</word>
          <word xMin=

In [ ]:
from pprint import pprint

pprint(vizlink.pdf_word_list[:5])
pprint(vizlink.coordinate_map.items()[:5])

### Step 3: Parse HTML

In [ ]:
# import os
# from snorkel.parser import CorpusParser
# from snorkel.parser import HTMLParser
# from snorkel.parser import OmniParser

# html_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_html/'
# filename = 'bc546-d'
# html_file = html_path + filename + '.html'
# doc_parser = HTMLParser(path=html_file)
# context_parser = OmniParser()
# cp = CorpusParser(doc_parser, context_parser, max_docs=100) 

# %time corpus = cp.parse_corpus(name='Hardware', session=session)

import os
from snorkel.models import Corpus
from snorkel.utils import get_ORM_instance

os.system('cp snorkel.db\ corpus snorkel.db');
corpus = get_ORM_instance(Corpus, session, 'Hardware')

In [ ]:
%time vizlink.extract_html_words(corpus.documents[0])

In [ ]:
from pprint import pprint
pprint(vizlink.html_word_list[:5])

### Step 4: Visual Linking

In [ ]:
%time vizlink.link_lists(searchMax=200)

In [ ]:
pprint(vizlink.links.items()[:5])

In [ ]:
phrase = corpus.documents[0].phrases[15]
print phrase
print phrase.page
print phrase.top
print phrase.bottom
print phrase.left
print phrase.right

In [ ]:
import pandas as pd
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

matches = vizlink.display_links()
print_full(matches[:200])

### Step 5: Viewing Results

In [ ]:
import os
from collections import defaultdict
from snorkel.models import Corpus, candidate_subclass
from snorkel.matchers import RegexMatchSpan, Union
from snorkel.candidates import CandidateExtractor
from snorkel.utils import get_ORM_instance
from hardware_utils import OmniNgramsPart, OmniNgramsTemp, get_gold_dict

# Candidate Type
Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])
    
# CandidateSpaces
part_ngrams = OmniNgramsPart(parts_by_doc=None, n_max=3) # NOTE: no part linking right now
temp_ngrams = OmniNgramsTemp(n_max=2)

# Matchers
eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

temp_matcher = RegexMatchSpan(rgx=r'1[4-6]0', longest_match_only=False)

# Throttler
part_throttler = lambda x: x[0].parent.page == x[1].parent.page

# Extractor
ce = CandidateExtractor(Part_Temp, 
                        [part_ngrams, temp_ngrams], 
                        [parts_matcher, temp_matcher], 
                        throttler=part_throttler)

# Extract
for corpus_name in ['Hardware']:
    corpus = get_ORM_instance(Corpus, session, corpus_name)
    print "Extracting Candidates from %s" % corpus
    %time candidates = ce.extract(\
        corpus.documents, corpus_name + ' Candidates', session)
    session.add(candidates)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

In [ ]:
c = candidates[5]
print c.part
print c.part.char_start, c.part.char_end
print c.part.get_word_start()
print c.part.get_word_end()
print c.part.parent.page
print c.part.get_attrib_tokens('top')
print c.part.get_attrib_tokens('bottom')
print c.part.get_attrib_tokens('left')
print c.part.get_attrib_tokens('right')

In [ ]:
vizlink.display_candidates(candidates, page_num=2, display=True)

In [ ]:
# from visual_linking import display_boxes, get_box

# boxes = []
# for c in candidates:
#     boxes.append(get_box(c.part))
# boxes = list(set(boxes))
# display_boxes(pdf_file, boxes, page_num=2)

### Display Ordering of PDF Word List 

Display Ordering on a black page - Not very easy to display because of superposition

In [ ]:
# import numpy as np
# import cv2
# import math

# page_num = 2
# page_height = 792
# page_width = 612
# img = np.ones((page_height,page_width,3))*255
# font = cv2.FONT_HERSHEY_SIMPLEX
# letter_width = 3
# i = 0
# for word_id, _ in pdf_word_list:
#     if word_id[0] == page_num:
#         i += 1
#         _, top, left, bottom, right = coordinate_map[word_id]
#         cv2.rectangle(img, (left, top), (right, bottom), (0,255,0), 1)
#         cv2.putText(img, 
#                     str(i), 
#                     ((left + right)/2 - letter_width*int(math.ceil(math.log10(i))), 
#                     bottom + (top - bottom)/4), 
#                     font, 
#                     0.3, 
#                     (255,0,0), 
#                     1)
# cv2.imshow('PDF Word List Order',img)
# cv2.waitKey() # press any key to exit the opencv output 
# cv2.destroyAllWindows() 

The end.